# Computing Relative Wealth Index Zonal Statistics of Lacuna Cities

This notebook is computes the relative wealth index statistics (mean, median, max, min, and standard deviation) of each zone for Lacuna Cities throughout time.

## Setup

In [2]:
import geopandas as gpd
import pandas as pd
import geowrangler.vector_zonal_stats as vzs

/home/cczablan1/projects/unicef-ai4d-poverty-mapping/env/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Computing Geowrangler Vector Zonal Statistics

In [ ]:
# Input and Output Data Directory
INPUT_DIR = '/path/to/your/data'
OUTPUT_DIR = '/path/to/your/output'

In [3]:
# Open Lacuna Cities Shapefile (barangay boundary) as Geodataframe
lacuna_cities_gdf = gpd.read_file(f"{INPUT_DIR}/target_admin_bounds.shp")

# List all Lacuna Cities 
lacuna_cities_list = ["Legazpi City", "Mandaue City", "Davao City", "Iloilo City", "Tacloban City",
                    "City of Muntinlupa", "City of Mandaluyong", "City of Navotas",
                    "Cagayan de Oro City", "Dagupan City", "Palayan City", "Zamboanga City"]


In [6]:
# Create an empty list that we iteratively house a dataframe containing the RWI Zonal Statistics for a specific year  
lacuna_RWI_stats_list = []

# Create a loop that processes RWI data from 2016 to 2022
for year in range(2021, 2023):
    counter = str(year)+"_RWI_"
    print(counter)
    
    # Open RWI Data and remove non-Lacuna city features.
    rollout_aoi = gpd.read_file(f"{INPUT_DIR}/[{year}]_2023-02-21-ph-rollout-output.geojson")
    lacuna_rollout_aoi = rollout_aoi[rollout_aoi['ADM3_EN'].isin(lacuna_cities_list)]

    # Compute Zonal Statistics from RWI Data with the Lacuna City Geodataframe (per barangay) as the zones
    stats_list = ["mean", "median", "min", "max", "std"]
    lacuna_RWI_stats = vzs.create_zonal_stats(
        lacuna_cities_gdf,
        lacuna_rollout_aoi,
        overlap_method="intersects",
        aggregations=[{
                "column": "Predicted Relative Wealth Index",
                "func": stats_list,
                "output": [counter + stat for stat in stats_list],
            }],
    )

    lacuna_RWI_stats_list.append(lacuna_RWI_stats)

2021_RWI_
2022_RWI_


In [7]:
# Check
lacuna_RWI_stats.head(3)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,2022_RWI_mean,2022_RWI_median,2022_RWI_min,2022_RWI_max,2022_RWI_std
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",0.570387,0.570389,0.541520,0.620761,0.022070
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",0.691508,0.732137,0.558810,0.755687,0.070484
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Pantal,PH015518022,"POLYGON ((120.34737 16.06009, 120.34761 16.060...",0.657205,0.648928,0.556726,0.755283,0.071617


In [7]:
# Concatenate List containing Lacuna Cities' Yearly RWI Zonal Statistics Geodataframe and group by Barangay.
lacuna_RWI_stats_concat = pd.concat(lacuna_RWI_stats_list).groupby('ADM4_PCODE').first()
lacuna_RWI_stats_concat

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,geometry,2016_RWI_mean,2016_RWI_median,...,2019_RWI_mean,2019_RWI_median,2019_RWI_min,2019_RWI_max,2019_RWI_std,2020_RWI_mean,2020_RWI_median,2020_RWI_min,2020_RWI_max,2020_RWI_std
ADM4_PCODE,,,,,,,,,,,,,,,,,,,,,
PH015518001,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Bacayao Norte,"POLYGON ((120.34108 16.03818, 120.34117 16.038...",0.668600,0.670609,...,0.667605,0.668669,0.635550,0.733501,0.033219,0.692400,0.680741,0.656662,0.731254,0.026909
PH015518002,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Bacayao Sur,"POLYGON ((120.34784 16.02607, 120.34767 16.026...",0.671211,0.684132,...,0.671020,0.682731,0.579508,0.709324,0.047200,0.712864,0.715705,0.680457,0.749918,0.028376
PH015518003,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Barangay II (Nueva),"POLYGON ((120.34098 16.04190, 120.34106 16.041...",0.633512,0.633512,...,0.628263,0.628263,0.627630,0.628897,0.000896,0.708711,0.708711,0.688884,0.728539,0.028041
PH015518004,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Barangay IV (Zamora),"POLYGON ((120.33584 16.04322, 120.33611 16.042...",0.692327,0.713386,...,0.691942,0.713427,0.628897,0.733501,0.055513,0.706203,0.698472,0.688884,0.731254,0.022218
PH015518006,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Bolosan,"POLYGON ((120.36857 16.05407, 120.36896 16.053...",0.603005,0.607531,...,0.602667,0.609345,0.516139,0.659507,0.045419,0.631495,0.615092,0.603756,0.686050,0.032368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PH137603005,National Capital Region,PH130000000,"NCR, Fourth District",PH137600000,City of Muntinlupa,PH137603000,Poblacion,"POLYGON ((121.05303 14.39260, 121.05317 14.392...",0.656875,0.658930,...,0.655666,0.656264,0.566992,0.731564,0.044372,0.684381,0.686965,0.542254,0.785730,0.051910
PH137603006,National Capital Region,PH130000000,"NCR, Fourth District",PH137600000,City of Muntinlupa,PH137603000,Putatan,"POLYGON ((121.03526 14.40867, 121.03691 14.408...",0.700231,0.704654,...,0.700072,0.704443,0.597785,0.789809,0.039325,0.718403,0.717413,0.659306,0.785730,0.031006
PH137603007,National Capital Region,PH130000000,"NCR, Fourth District",PH137600000,City of Muntinlupa,PH137603000,Sucat,"POLYGON ((121.05173 14.46746, 121.05177 14.467...",0.670732,0.671203,...,0.670472,0.671669,0.570975,0.739684,0.047951,0.686394,0.688559,0.603092,0.741239,0.032380


In [11]:
# Export GEOJSON
lacuna_RWI_stats_concat.to_file(f"{OUTPUT_DIR}/lacuna_cities_RWI_2016_2022.json", driver="GeoJSON")

In [12]:
# Export CSV
lacuna_RWI_stats_concat.drop(['geometry'], axis=1).to_csv(f'{OUTPUT_DIR}/lacuna_cities_RWI_2016_2022.csv', encoding='utf-8', index=False)